# Question 1

Given two strings s and t, determine whether some anagram of t is a substring of s. For example: if s = "udacity" and t = "ad", then the function returns True. Your function definition should look like: question1(s, t) and return a boolean True or False.

### Solution 1
In this solution, I build a dictionary based on input string t, where keys are characters from string t, and values are the count of each character. Because the character order doesn't matter for string t, we build the dictionary for quick look up. We then loop through every same length substrings in string s, and build counter dictionary for each substring and compare this substring dictionary with the previously built target dictionary. If we find a matching dictionary, we have found a substring that's an anagram of t. The time complexity is O(m*n) given m is the length of s, and n is the length of t.

In [243]:
from collections import Counter
def question1(s, t):
    t_len = len(t)
    s_len = len(s)
    
    if t_len > s_len:
        return False
    
    # O(n) operation to put everything in to counter dict
    t_counter = Counter(t)
    
    len_diff = s_len-t_len+1
    
    # O(m*n) operation
    for i in range(len_diff):
        sub_string = s[i:i+t_len]
        s_counter = Counter(sub_string)

        # Verify two counters have same number of entries
        if len(s_counter) != len(t_counter):
            continue
        
        matched_counters = True
        for c in s_counter:
            if (c not in t_counter) or (s_counter[c]!=t_counter[c]):
                matched_counters = False
                break
        
        if matched_counters:
            return True
        
    return False
            

In [248]:
print(question1('abzqabeea','aeaeb'))
print(question1('acdeabb','dca'))
print(question1('acdeabb','xx'))
print(question1('acdeabb','abde'))
print(question1('ebad','abde'))

True
True
False
True
True


### Solution 2
In the previos solution, we have to build a dictionary and do a dictionary comparison for every substring. In this solution, instead of converting every substring into another counter dictionary, we build a difference dictionary to track the difference between current substring and the target dictionary. When moving from one substring to another substring, we can simply update the difference dictionary. Wehn the dictionary becomes empty, we have a match. This solution removes the dictionary comparison to achieve O(m) time complexity.

In [251]:
from collections import Counter
def question1(s, t):
    t_len = len(t)
    s_len = len(s)
    
    if t_len > s_len:
        return False
    
    # O(n) operation to put everything in to counter dict
    t_counter = Counter(t)

    # Build diff counter for the very first substring
    first_sub_string = s[:t_len]
    s_counter = Counter(first_sub_string)
    diff_counter = Counter()
    
    # Diff_counter = substring_s_counter - t_counter
    # positive count means current substring has extra character
    # negative count means current substring is missing character 
    for key in s_counter:
        if (key in s_counter) and (key not in t_counter):
            diff_counter[key] = s_counter[key]
        else:
            diff_counter[key] = s_counter[key] - t_counter[key]
    for key in t_counter:
        if (key in t_counter) and (key not in s_counter):
            diff_counter[key] = -1*t_counter[key]
            
    # Check if this first substring is already a match
    keys = list(diff_counter.keys())
    for key in keys:
        if diff_counter[key] == 0:
            # Remove all zero count character
            diff_counter.pop(key)
    if len(diff_counter)==0:
        return True
    
    # Loop through s, update and check diff_counter
    for i in range(1, s_len-t_len+1):
        removed_c = s[i-1]
        added_c = s[i+t_len-1]
        
        diff_counter[removed_c] -= 1
        if diff_counter[removed_c] == 0:
            diff_counter.pop(removed_c)
        
        diff_counter[added_c] += 1
        if diff_counter[added_c] == 0:
            diff_counter.pop(added_c)
        
        if len(diff_counter) == 0:
            return True
        
    return False
            

In [252]:
print(question1('abzqabeea','aeaeb'))
print(question1('acdeabb','dca'))
print(question1('acdeabb','xx'))
print(question1('acdeabb','abde'))
print(question1('ebad','abde'))

True
True
False
True
True


# Question 2

Given a string a, find the longest palindromic substring contained in a. Your function definition should look like question2(a), and return a string.

### Solution 1
Define a recursoin function findPalindrom(str), which takes in a string and returns the longest palindromic substring within it. If the left end of the string is the same character of the right end of the string, and the substring between it is a palindromic string, the whole string is a palindromic string. Otherwise, we call the same findPalindrom function for these three substrings: a) remove left end of current string b) remove right end of current string c) remove both end of current string. And see which one returns the longest palindromic string and return the palindromic string.

In [267]:
def findPalindrom(str):
    if len(str) == 1:
        return str
    elif len(str) == 2:
        if str[0] == str[1]:
            return str
        else:
            return ''
    elif len(str) == 3:
        if str[0] == str[2]:
            return str
        elif str[0] == str[1]:
            return str[:2]
        elif str[1] == str[2]:
            return str[1:]
        else:
            return ''
    else:
        if (str[0] == str[-1]):
            subStr3 = findPalindrom(str[1:-1])
            if (len(subStr3) == len(str)-2):
                return str
        subStr1 = findPalindrom(str[:-1])
        subStr2 = findPalindrom(str[1:])
        if len(subStr1) >= len(subStr2):
            return subStr1
        else:
            return subStr2

def question2(a):
    return findPalindrom(a)

In [326]:
print(question2('a'))
print(question2('ab'))
print(question2('aba'))
print(question2('abbba'))
print(question2('adbbca'))
print(question2('abbbca'))
print(question2('adbbba'))
print(question2('abaab'))
print(question2('abbab'))

a
b
aba
abbba
bb
bbb
bbb
baab
abba


### Solution 2
In the previous solution, we repetely calculate same substring for many times. To get a O(n^2) solution, I created a 2D array to store the longest palindromic substring so we don't need to recalculate the same thing over and over again. The recusion formula is still the same.

In [328]:
def question2(a):
    n = len(a)

    # Create a n x n array, where n is the length of the input str
    # palindrom[i][j] = longest palindrom from str[i] to str[j]
    palindrom = [['']*n for i in range(n)]
    
    # Init diagonal line, where i == j
    for i in range(n):
        palindrom[i][i] = a[i]

    # 2 layer for loops to build up the array from bottom right to top right
    # Our target is the top right corner palindrom[0][n]
    for i in reversed(range(n-1)):
        for j in range(i+1,n):
            # If the left end char is the same as right end char, and the string in between 
            # is a palindrom, the whole string is a palindrom
            if (a[i] == a[j]) and (len(palindrom[i+1][j-1]) == j-i-1):
                palindrom[i][j] = a[i:j+1]
                
            # Otherwise, find out the max len palindrome from palindrom[i][j-1] and palindrom[i+1][j] 
            else:
                if len(palindrom[i][j-1]) > len(palindrom[i+1][j]):
                    palindrom[i][j] = palindrom[i][j-1]
                else:
                    palindrom[i][j] = palindrom[i+1][j]

                #lenA = len(palindrom[i][j-1])
                #lenB = len(palindrom[i+1][j])
                #lenC = len(palindrom[i+1][j-1])
                #max_len = max([lenA, lenB, lenC])
                #if max_len == lenA:
                #    palindrom[i][j] = palindrom[i][j-1]
                #elif max_len == lenB:
                #    palindrom[i][j] = palindrom[i+1][j]
                #else:
                #    palindrom[i][j] = palindrom[i+1][j-1]

    return palindrom[0][n-1]

In [329]:
print(question2('a'))
print(question2('ab'))
print(question2('aba'))
print(question2('abbba'))
print(question2('adbbca'))
print(question2('abbbca'))
print(question2('adbbba'))
print(question2('abaab'))
print(question2('abbab'))

a
b
aba
abbba
bb
bbb
bbb
baab
abba


# Question 3

Given an undirected graph G, find the minimum spanning tree within G. A minimum spanning tree connects all vertices in a graph with the smallest possible total weight of edges. Your function should take in and return an adjacency list structured like this:

{'A': [('B', 2)],


 'B': [('A', 2), ('C', 5)], 
 
 
 'C': [('B', 5)]}

Vertices are represented as unique strings. The function definition should be question3(G)

### Solution
I implemented classic prim's algorithm for minimum spanning tree. The algorithm maintains a distance for each node it can reach currently, and add the minimum distance edge everytime.

In [332]:
def question3(G):
    int_max = 1000000
    included_nodes = []
    included_edges = []
    result = dict()
    
    # Init not_included_nodes dictionary. The keys are all "not yet included" nodes, and the value is 
    # the current shortest distance from "included nodes" to this node, and from which "included node"
    # gives the shortest distance.
    not_included_nodes = dict()
    for node in G:
        not_included_nodes[node] = {'dist':int_max, 'from':''}
        
    # Add one node as starting point
    first_node = list(G.keys())[0]
    not_included_nodes[first_node]['dist'] = 0
    not_included_nodes[first_node]['from'] = ''
        
    while len(included_nodes) != len(G):
        # Search for shortest distance from all "not included" nodes
        min_dist = int_max
        for node in not_included_nodes:
            if not_included_nodes[node]['dist'] < min_dist:
                min_dist_node = node
                min_dist = not_included_nodes[node]['dist']
        
        # Add the node to "included nodes"
        included_nodes.append(min_dist_node)
        
        # Add the edges to output dictionary
        from_node = not_included_nodes[min_dist_node]['from']
        if from_node == '':
            # For the very first node, don't add the edge into result
            result[min_dist_node] = []
        else:
            result[min_dist_node] = [(from_node, min_dist)]
            result[from_node].append((min_dist_node, min_dist))
        
        # Remove node from not_included_nodes
        not_included_nodes.pop(min_dist_node)
        
        # Update distances in not_included_nodes for all the connect nodes from this newly added node
        for node, dist in G[min_dist_node]:
            if (node not in included_nodes) and (dist < not_included_nodes[node]['dist']):
                not_included_nodes[node]['dist'] = dist
                not_included_nodes[node]['from'] = min_dist_node

    return result
        

In [333]:
Input = {'A': [('B', 2),('C',10),('E',3)], 'B': [('A', 2), ('C', 5)], 'C': [('A',10),('B', 5),('D',7)],'D':[('C',7),('E',11)],'E':[('D',11),('A',3)]}
print(question3(Input))

{'A': [('B', 2), ('E', 3)], 'B': [('A', 2), ('C', 5)], 'E': [('A', 3)], 'C': [('B', 5), ('D', 7)], 'D': [('C', 7)]}


# Question 4

Find the least common ancestor between two nodes on a binary search tree. The least common ancestor is the farthest node from the root that is an ancestor of both nodes. For example, the root is a common ancestor of all nodes on the tree, but if both nodes are descendents of the root's left child, then that left child might be the lowest common ancestor. You can assume that both nodes are in the tree, and the tree itself adheres to all BST properties. The function definition should look like question4(T, r, n1, n2), where T is the tree represented as a matrix, where the index of the list is equal to the integer stored in that node and a 1 represents a child node, r is a non-negative integer representing the root, and n1 and n2 are non-negative integers representing the two nodes in no particular order. For example, one test case might be

question4(<br>
[[0, 1, 0, 0, 0],<br>
[0, 0, 0, 0, 0],<br>
[0, 0, 0, 0, 0],<br>
[1, 0, 0, 0, 1],<br>
[0, 0, 0, 0, 0]],<br>
3, 1, 4)
          
and the answer would be 3.

### Solution
Given the ordering chracteristic of binary search tre, we can simply start with comparing n1, n2 with current root, and decide to go to left sub tree or right subtree. Continue the same logic until n1 is on one side of the node and n2 is on the other side of the node, or n1 or n2 is current node. 

In [336]:
def question4(T, r, n1, n2):
    while 1:
        prev_r = r
        # We've found the LCA if n1 == root or n2 == rot or n1 < root and n2 > root
        if (n1 == r) or (n2 == r) or (n1 < r and n2 > r):
            return r
        # Both n1 and n2 are on the left side of the root, find left child and set it as new root
        elif (n1 < r) and (n2 < r):
            for i in range(r):
                if T[r][i] == 1:
                    r = i
                    break
        else:
            for i in range(r+1,len(T)):
                if T[r][i] == 1:
                    r = i
                    break

        # We couldn't find the next root to go. The tree passed in is not valid.
        if prev_r == r:
            break

    print("Error: Invalid BST")
    return None
            
                    

In [337]:
print(question4(
    [[0, 0, 0, 0, 0],
    [1, 0, 0, 1, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 1, 0, 0, 0]],
    4, 3, 2))

print(question4([[0, 1, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0],
[1, 0, 0, 0, 1],
[0, 0, 0, 0, 0]],
3,1,4))

3
3


# Question 5

Find the element in a singly linked list that's m elements from the end. For example, if a linked list has 5 elements, the 3rd element from the end is the 3rd element. The function definition should look like question5(ll, m), where ll is the first node of a linked list and m is the "mth number from the end". You should copy/paste the Node class below to use as a representation of a node in the linked list. Return the value of the node at that position.

class Node(object):<br>
  def __init__(self, data):<br>
    self.data = data<br>
    self.next = None<br>

### Solution
Set two pointers, one is at the begining of the linked list and another has already moved m-th positions. Then we move both pointers toward the end until the secound one reaches the end of linked list. At that point, the first pointer will be m th node from the end.

In [345]:
class Node(object):
  def __init__(self, data):
    self.data = data
    self.next = None
    
def question5(ll, m):
    ptr_back = ll
    ptr_front = ll
    for i in range(m-1):
        if ptr_front.next:
            ptr_front = ptr_front.next
        else:
            print('Error: linked list too short')
            return None
    
    while ptr_front.next:
        ptr_back = ptr_back.next
        ptr_front = ptr_front.next

    return ptr_back.data
        

In [346]:
N1 = Node(1)
N2 = Node(2)
N3 = Node(3)
N4 = Node(4)
N5 = Node(5)
ll = N1
N1.next = N2
N2.next = N3
N3.next = N4
N4.next = N5

print(question5(ll, 3))
print(question5(ll, 6))

3
Error: linked list too short
None
